In [2]:
import pandas as pd #used for data manipulation and analysis
from sklearn.model_selection import train_test_split #split dataset for training and testing
import math #python library for all mathematical functions
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import random #generate random numbers or selections
from sklearn.manifold import TSNE #TSNE class tool to visualize high-dimensional data
#t-sne is a dimensionality reduction technique used in data visualization and analysis
import seaborn as sns #for creating statistical graphics
import plotly.express as px
import torch.optim as optim
from tqdm import tqdm #used to create progress bars
import torch
import torch.nn as nn
import torch.nn.functional as F
from  torch.utils.data import Dataset, DataLoader #for handling and loading data when working with PyTorch
import cv2 #pip install opencv-python #image processing library
import urllib #provides functions for working with URLs and web requests
from colabcode import ColabCode

# Variational Autoencode Implementation (VAE)

- A type of neural network architecture used for generative modeling and dimensionality reduction
- A type of artificial neural network and data compression


1. **Autoencoder**: Think of an autoencoder as a system that tries to compress information. It has two main parts:
   - **Encoder**: This part takes input data (like images, text, or any kind of information) and converts it into a compressed representation, often called a "latent space" or "encoding." This encoding is a condensed version of the input data.
   - **Decoder**: The decoder takes this compressed representation and tries to recreate the original input data from it. It's like a reverse process of the encoder.

2. **Variational**: The "variational" part of VAE means that it introduces a level of randomness into the encoding process. Instead of producing a single fixed encoding for a given input, VAE generates a probability distribution of possible encodings. This adds a level of uncertainty to the process.


- **Encoding with Variability**: When you feed data into the VAE, it doesn't just produce one fixed encoding. Instead, it produces a range of possible encodings, represented as a probability distribution. This variability is useful because it allows the VAE to capture the inherent uncertainty in real-world data.

- **Sampling**: From this probability distribution, a point is randomly sampled. This sampled point becomes the encoding for the input data. This random sampling is a key feature of VAE and is what makes it "variational."

- **Decoding**: The sampled encoding is then passed through the decoder to generate a reconstructed version of the original input data.

- **Training**: During the training process, VAE tries to minimize the difference between the original data and the reconstructed data. It also tries to make the distribution of encodings as close to a standard Gaussian distribution as possible. This encourages the model to learn a meaningful and structured latent space representation of the data.

So, in essence, a Variational Autoencoder is a neural network that can both compress data into a meaningful latent space and generate data from points in that latent space while introducing a level of randomness. This randomness makes it a powerful tool for various applications, including image generation, data denoising, and more, as it can capture the uncertainty and diversity present in real-world data.

How it works:
  encoder -> latent space -> decoder -> variational part

**Encoder**
  - takes the picture and tries to learn a compressed representation of it
  - looks for essential features (e.g eyes, ears, snout, etc.)
  - hidden code

**Latent Space**
  - special space where each point represents a unique feature decoder
  - a place for hidden codes from the encoder

**Decoder**
 - the decoder has learned how to tranform these latent codes into recognizable images
 - variational part
    - in this part, it makes VAE special; it can generate slight difference from the original

Directory Structure
- input
    - data
- outputs
- src
    - model.py
    - train.py



In [ ]:
class VariationalAutoencoder(nn.Module):
  #constructor to create a VAE object, allowing parameters num_feature and num_dim
  def __init__(self, num_features = 8, num_dim = 784): #28x28 pixels
    super(VariationalAutoencoder, self).__init__()
    self.num_features = num_features
    self.num_dim = num_dim

    #creates a neural network layer that will be used for encoding input data
    self.encoder_layer_1 = nn.Linear(in_features = self.num_dim, out_features = 512)

    #creates another encoding layer that takes 512 features from the previous layer
    #and produces num_features * 2
    self.encoder_layer2 = nn.Linear(in_features = 512, out_features = (self.num_features*2))

    #code responsible for reconstruction of data from the learned latent space
    self.decoder_layer_1 = nn.Linear(in_features = self.num_features, out_features = 512)
    self.decoder_layer_2 = nn.Linear(in_features = 512, out_features = (self.num_dim))


  def reparameterize(self, mu, log_var):
    '''
    :param mu: mean from the encoder's latent space
        - is like the desired average or center point inside the box
    :param log_var: log variance from the encoder's latent space
        - is like how spread out you want your random stuff to be inside the box(higher values mean more spread)
    '''
    std = torch.exp(0.5 * log_var) #standard deviation
    eps = torch.randn_like(std) #randn_like as we need the same size
    sample = mu + (eps * std) #sampling as if coming from the input space

    return sample

  def encode(self, x):
    #encoding
    x = F.relu(self.encoder_layer_1(x))
    #relu (rectified linear unit) - a mathematical function commonly used in AI neural network
    #ReLu(x) = max(0,x)
    x = self.encoder_layer_2(x).view(-1, 2, self.num_features)

    #get mu and log_var
    mu = x[:, 0, :]
    log_var = x[:, 1, :]

    z = self.reparameterize(mu, log_var)

    return z, mu, log_var


